# Model M32


|Markets   |    A   |    B   |     W   |   TL  |  TK  |    |   CONS|
|----------|--------|--------|---------|-------|------|----|------ |
|PX        |  120   |        |  -120   |       |      |    |       |    
|PY        |        |  120   |  -120   |       |      |    |       |        
|PW        |        |        |   340   |       |      |    |  -340 | 
|PLS       |  -48   |  -72   |         |  120  |      |    |       |     
|PKS       |  -72   |  -48   |         |       |   120|    |       |       
|PL        |        |        |  -100   | -100  |      |    |200    |     
|PK        |        |        |         |       |  -100|    |100    |      
|TAX       |        |        |         |  -20  |   -20|    |  40   |    


In [18]:
using MPSGE
using DataFrames

In [19]:
M32 = MPSGEModel()


txl0 = 20/100
txk0 = 20/100

@parameters(M32, begin
    TXL, txl0, (description = "Labor income tax rate",)
    TXK, txk0, (description = "Captial income tax rate",)
end)

@sectors(M32, begin
    X,	(description = " Activity level for sector X",)
    Y,	(description = " Activity level for sector Y",)
    W,	(description = " Activity level for sector W (Hicksian welfare index)",)
    TL,	(description = " Supply activity for L",)
    TK,	(description = " Supply activity for K",)
end)


@commodities(M32, begin
    PX,	(description = " Price index for commodity X",)
    PY,	(description = " Price index for commodity Y",)
    PL,	(description = " Price index for primary factor L (net of tax)",)
    PK,	(description = " Price index for primary factor K (net of tax)",)
    PLS,	(description = " Price index for primary factor L (gross of tax)",)
    PKS,	(description = " Price index for primary factor K (gross of tax)",)
    PW,	(description = " Price index for welfare (expenditure function)",)
end)

@consumer(M32, CONS, description = "Income level for consumer CONS")

@production(M32, X, [s=1,t=0], begin
    @output(PX, 120, t)
    @input(PLS, 40, s, reference_price = 1.2)
    @input(PKS, 60, s, reference_price = 1.2)
end)

@production(M32, Y, [s=1,t=0], begin
    @output(PY, 120, t)
    @input(PLS, 60, s, reference_price = 1.2)
    @input(PKS, 40, s, reference_price = 1.2)
end)

@production(M32, TL, [s=0,t=0], begin
    @output(PLS, 100, t , reference_price = 1.2)
    @input(PL, 100, s, taxes = [Tax(CONS, TXL)])
end)

@production(M32, TK, [s=0,t=0], begin
    @output(PKS, 100, t, reference_price = 1.2)
    @input(PK, 100, s, taxes = [Tax(CONS, TXK)])
end)

@production(M32, W, [s=.7, a=>s=1, t=0], begin
    @output(PW, 340, t)
    @input(PX, 120, a)
    @input(PY, 120, a)
    @input(PL, 100, s)
end)

@demand(M32, CONS, begin
    @final_demand(PW, 340)
    end,begin
    @endowment(PL, 200)
    @endowment(PK, 100)
end);

# Benchmark

In [20]:
fix(PW, 1)

set_start_value(PLS, 1.2)
set_start_value(PKS, 1.2)

solve!(M32; cumulative_iteration_limit=0)
generate_report(M32)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_1A.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 12

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Row,var,value,margin
,GenericV…,Float64,Float64
1,W,1.0,0.0
2,Y,1.0,0.0
3,X,1.0,0.0
4,PL,1.0,0.0
5,PW,1.0,0.0
6,PKS,1.2,0.0
7,PY,1.0,0.0
8,PX,1.0,0.0
9,PLS,1.2,0.0


# Counterfactual

Counterfactual with taxes shifted to the factor which is in fixed supply

In [21]:
set_silent(M32)

df = DataFrame([[],[]], [:welfare,:labor_supply])

for s∈1:5
    set_value!(TXL, .25 - .05*s)
    set_value!(TXK, .15 + .05*s)

    solve!(M32)

    push!(df, [value(W), value(TL)])
end
    

In [22]:
df

Row,welfare,labor_supply
,Any,Any
1,1.0,1.0
2,1.00084,1.01609
3,1.00148,1.03287
4,1.00189,1.05038
5,1.00203,1.06869
